<a href="https://colab.research.google.com/github/dibdin/malebirthcontrol/blob/main/Part1_SPEM1_data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computational Drug Discovery [Part 1] Download Bioactivity Data**

Original code credit: Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)


---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for SPEM1**

In [ ]:
# Target search for SPEM1
target = new_client.target
target_query = target.search('spermatid maturation protein 1')
targets = pd.DataFrame.from_dict(target_query)
targets

### **Select and retrieve bioactivity data for *SPEM1* 


We will assign the the target protein, *SPEM1* to the ***selected_target*** variable 

In [ ]:
selected_target = targets.target_chembl_id[2]
selected_target

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

In [ ]:
df.standard_type.unique()

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data_spem1.csv**.

In [ ]:
df.to_csv('bioactivity_data_spem1.csv', index=False)

## **Copying files to Google Drive**

Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Next, we create a **data** folder in our **Colab Notebooks** folder on Google Drive.

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/databirthcontrol"

In [ ]:
! cp bioactivity_data_spem1.csv "/content/gdrive/My Drive/Colab Notebooks/databirthcontrol"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/databirthcontrol"

Let's see the CSV files that we have so far.

In [ ]:
! ls

Taking a glimpse of the **bioactivity_data_spem1.csv** file that we've just created.

In [ ]:
! head bioactivity_data_spem1.csv

## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [ ]:
df2 = df[df.standard_value.notna()]
df2

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Iterate the *molecule_chembl_id* to a list**

In [ ]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

### **Iterate *canonical_smiles* to a list**

In [ ]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

### **Iterate *standard_value* to a list**

In [ ]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

### **Combine the 4 lists into a dataframe**

In [ ]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [ ]:
df3

### **Alternative method**

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[selection]
df3

In [ ]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

Saves dataframe to CSV file

In [ ]:
df3.to_csv('bioactivity_preprocessed_data_spem1.csv', index=False)

In [ ]:
! ls -l

Let's copy to the Google Drive

In [ ]:
! cp bioactivity_preprocessed_data_spem1.csv "/content/gdrive/My Drive/Colab Notebooks/databirthcontrol"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/databirthcontrol"

---